In [1]:
!unzip /content/test.zip
!unzip /content/train.zip

Archive:  /content/test.zip
   creating: test/
   creating: test/flamingo/
  inflating: test/flamingo/image_0022.jpg  
  inflating: test/flamingo/image_0024.jpg  
  inflating: test/flamingo/image_0041.jpg  
  inflating: test/flamingo/image_0043.jpg  
  inflating: test/flamingo/image_0047.jpg  
  inflating: test/flamingo/image_0050.jpg  
  inflating: test/flamingo/image_0051.jpg  
  inflating: test/flamingo/image_0059.jpg  
   creating: test/pizza/
  inflating: test/pizza/image_0024.jpg  
  inflating: test/pizza/image_0035.jpg  
  inflating: test/pizza/image_0037.jpg  
  inflating: test/pizza/image_0038.jpg  
  inflating: test/pizza/image_0040.jpg  
  inflating: test/pizza/image_0046.jpg  
Archive:  /content/train.zip
   creating: train/
   creating: train/flamingo/
  inflating: train/flamingo/image_0001.jpg  
  inflating: train/flamingo/image_0002.jpg  
  inflating: train/flamingo/image_0003.jpg  
  inflating: train/flamingo/image_0004.jpg  
  inflating: train/flamingo/image_0005.jpg  

--- *LIBRARIES* ----------------------------------------------------------------

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
import glob
from sklearn.utils import shuffle
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

---*TRAIN IMAGE IMPORT*---------------------------------------------------------

In [2]:
train_image_vectors = []
t = []

path_train_files = glob.glob("/content/train/**/*.jpg", recursive = True)
train_files = []

i = 0
for file in path_train_files:
    train_files.append(file)
    train_image = cv2.imread(file)
    label = train_files[i].split('/')[-2]

    #cv2_imshow(train_image)
    #print(train_image.shape)
    train_image = cv2.resize(train_image, (128,128))
    #cv2_imshow(train_image)
    #print(train_image.shape)
    flat_train_image = train_image.flatten().astype(np.float16).reshape((1,-1))
    train_image_vectors.append(flat_train_image)
    t.append(label)
    i+=1


train_files = np.array(train_files).reshape((1,-1))
train_image_vectors = np.array(train_image_vectors)

print(t)
print(train_files.shape)
print(flat_train_image.shape)
print(train_image_vectors.shape)
# print(train_image_vectors)

['flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pi

---*TEST IMAGE IMPORT*----------------------------------------------------------

In [3]:
test_image_vectors = []
labels_test = []

path_test_files = glob.glob("/content/test/**/*.jpg", recursive = True)
test_files = []

i = 0
for file in path_test_files:
    test_files.append(file)
    test_image = cv2.imread(file)
    label = test_files[i].split('/')[-2]

    #cv2_imshow(train_image)
    #print(train_image.shape)
    test_image = cv2.resize(test_image, (128,128))
    #cv2_imshow(train_image)
    #print(train_image.shape)
    flat_test_image = test_image.flatten().astype(np.float16).reshape((1,-1))
    test_image_vectors.append(flat_test_image)
    labels_test.append(label)
    i+=1

test_files = np.array(test_files).reshape((1,-1))
test_image_vectors = np.array(test_image_vectors)

print(labels_test)
print(test_files.shape)
print(flat_test_image.shape)
print(test_image_vectors.shape)

# print(test_image_vectors)

['flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'flamingo', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza', 'pizza']
(1, 14)
(1, 49152)
(14, 1, 49152)


---*SHUFFLE DATA*--------------------------------------------------



In [4]:
combined_train_data = list(zip(train_image_vectors, t))
combined_test_data = list(zip(test_image_vectors, labels_test))

np.random.shuffle(combined_train_data)
np.random.shuffle(combined_test_data)

train_data, train_labels = zip(*combined_train_data)
test_data, test_labels = zip(*combined_test_data)

train_data = np.array(train_data)[:,0,:]
test_data = np.array(test_data)[:,0,:]

train_labels = list(train_labels)
test_labels = list(test_labels)

In [5]:
print(train_data.dtype)
print(test_data.dtype)

float16
float16


---*LABEL ENCODER*---------------------------------------------

In [6]:
label_encoder = LabelEncoder()

train_labels = label_encoder.fit_transform(train_labels)

print(train_labels)
# pizza --> 1
# flamingo --> 0
print(train_data.shape)

[0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 0
 0 1 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1
 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1]
(95, 49152)


---*CREATE .NPY FILE*-----------------------------------------

In [7]:
weights_path = "/content/weights.npy"
if not os.path.exists(weights_path):
  os.mknod(weights_path)

---*AKTIVASYON FONKS*----------------------------------

In [30]:
def normalize(x):
    x_min = np.min(x)
    x_max = np.max(x)
    x_normalized = (x - x_min) / (x_max - x_min)
    return x_normalized

def mish_normalized(x):
    x_normalized = normalize(x)
    result = x_normalized * tanh(softplus(x_normalized))
    return result

def tanh(x):
    exp_pos = np.exp(x)
    exp_neg = np.exp(-x)
    denominator = exp_pos + exp_neg
    denominator = np.clip(denominator, 1e-15, None)  # Küçük değerleri sınırla
    return (exp_pos - exp_neg) / denominator

def softplus(x):
    return np.log(1 + np.exp(-np.abs(x))) + np.maximum(x, 0)

def mish(x):
    result = x * tanh(softplus(x))
    if np.isnan(result).any():
        # NaN değeri tespit edildi,
        result = 0  # veya başka bir değer
    return result

def dmish(x):
    omega = np.exp(3*x) + 4*np.exp(2*x) + (6+4*x)*np.exp(x) + 4*(1 + x)
    delta = 1 + pow((np.exp(x) + 1), 2)
    derivative = np.exp(x) * omega / pow(delta, 2)
    if np.isnan(derivative).any():
        # NaN değeri tespit edildi,
        derivative = 0 # veya başka bir değer
    return derivative

---*TRAIN BIAS EKLEME*----------------------------------

In [9]:
new_column_train = np.ones((train_data.shape[0], 1))    # 1'lerden olusan yeni bir sutun bias icin
train_data = np.hstack((train_data, new_column_train))  # hstack ile iki diziyi yatay olarak birlestirir.
print(train_data.shape)
input_number_train = train_data.shape[1]

(95, 49153)


---*TRAIN PERCEPTRON FUNC*-----------------------------




In [31]:
def trainPerceptron(inputs, t, weights, rho, iterNo): # trin weights
  #threshold = 1.5
  #inputs = train_data
  #t = train_labels
  #bias = 1
  for _ in range(iterNo):
    for i in range(train_data.shape[0]):
      x = inputs[i]
      output = np.dot(x,weights) # ic carpim: iki vektoru carp sonuclari topla
      y = mish(output)
      error = t[i] - y
      dW = rho*error*dmish(y)*x
      dW = np.array(dW).reshape((-1,1))
      #print(dW.shape, weights.shape)

      #if np.isnan(dW).any():
      # print("NaN değeri tespit edildi.")
      weights += dW
  np.save('weights.npy', weights) # save weights

  return weights

---*TRAIN WEIGHTS*------------------------------------

In [11]:
# learning rate(rho) = 0.0001
# iteration number(iterNo) = 1000
# initial weight =
# initial_weights = 0.1 *(np.random.rand(len(train_data[0]), 1))
initial_weights = 0.001 * np.random.randn(len(train_data[0]), 1)
print(initial_weights.shape)

scaler = MinMaxScaler()
train_data_normalized = scaler.fit(train_data)
train_data_normalized = scaler.transform(train_data)

trained_weights = trainPerceptron(train_data, train_labels, initial_weights, 0.0001, 1000)

(49153, 1)


<ipython-input-8-52f90feb09e8>:13: RuntimeWarning: overflow encountered in exp
  exp_pos = np.exp(x)
<ipython-input-8-52f90feb09e8>:17: RuntimeWarning: invalid value encountered in divide
  return (exp_pos - exp_neg) / denominator


In [12]:
print(trained_weights)

[[315.30075597]
 [326.33964663]
 [373.85882303]
 ...
 [354.72017495]
 [410.87839092]
 [  2.52122558]]


---*TEST BIAS EKLEME*----------------------------------

In [13]:
new_column_test = np.ones((test_data.shape[0], 1))      # 1'lerden olusan yeni bir sutun bias icin
test_data = np.hstack((test_data, new_column_test))     # hstack ile iki diziyi yatay olarak birlestirir.


--- *TEST PERCEPTRON FUNC*----------------------------

In [32]:
test_data_normalized = scaler.transform(test_data)

def testPerceptron(sample_test, weights):
  # sample_test = test_data
  # weights = trained_weights
  labels = []
  for i in range(sample_test.shape[0]):
    predict = np.dot(sample_test[i], weights)
    print(predict)
    predicted_label = mish(predict)
    labels.append(predicted_label)
  return labels

In [15]:
print(test_data_normalized)
print(train_data_normalized)

[[0.29803922 0.36862745 0.78431373 ... 0.22745098 0.27058824 0.        ]
 [0.20392157 0.2627451  0.29803922 ... 1.         1.         0.        ]
 [0.07843137 0.21568627 0.25490196 ... 1.         1.         0.        ]
 ...
 [0.87843137 0.94509804 0.98039216 ... 0.57254902 0.83529412 0.        ]
 [1.         1.         0.99607843 ... 1.         1.         0.        ]
 [0.34117647 0.54901961 0.74901961 ... 0.49803922 0.61176471 0.        ]]
[[0.2        0.19215686 0.18823529 ... 0.81568627 0.85882353 0.        ]
 [0.16862745 0.20784314 0.19607843 ... 0.54509804 0.61176471 0.        ]
 [0.99607843 0.99607843 0.99607843 ... 0.99607843 0.99607843 0.        ]
 ...
 [0.1372549  0.14509804 0.1254902  ... 0.54509804 0.53333333 0.        ]
 [0.55686275 0.65098039 0.74509804 ... 0.42352941 0.41568627 0.        ]
 [0.30196078 0.19607843 0.11764706 ... 0.98431373 0.98431373 0.        ]]


---*TEST LABEL ENCODER*-----------------------------

In [16]:
test_labels = label_encoder.transform(test_labels)

In [26]:
weights = np.load('weights.npy') # trainPerceptron ile ogrenilmis weight verileri
sample_test = test_data_normalized
weights = weights

print(sample_test.shape)
print(weights.shape)

(14, 49153)
(49153, 1)


---*PREDICT*-------------------------------------

In [33]:
predicts = testPerceptron(sample_test, weights)

[6987130.511808]
[6580094.92834617]
[8038248.55839092]
[6526252.57474862]
[7863392.46611234]
[6206785.18395277]
[12262886.74381001]
[9831174.1436014]
[10505981.13473127]
[11250932.67148585]
[9541711.47762218]
[9499123.88820155]
[11391339.13576499]
[7577892.45414148]


<ipython-input-30-a0c66f529d81>:13: RuntimeWarning: overflow encountered in exp
  exp_pos = np.exp(x)
<ipython-input-30-a0c66f529d81>:17: RuntimeWarning: invalid value encountered in divide
  return (exp_pos - exp_neg) / denominator


---*PREADICT AND TEST LABELS*-------------------------------

In [34]:
predicts = np.array(predicts)
print("Predicted Labels: ", predicts)

print("Test Labels: ",test_labels)

Predicted Labels:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Test Labels:  [0 0 0 0 1 0 1 1 1 0 0 0 1 1]


---*ACCURACY SCORE*------------------------------------------

In [29]:
accuracy = accuracy_score(test_labels,predicts)
print(accuracy)

0.42857142857142855
